In [2]:
# Your code heree
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.feature_selection import chi2
from sklearn.exceptions import NotFittedError

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

from sklearn.metrics import mean_squared_error, r2_score

from pickle import dump

import os
import joblib

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/workspaces/fraud/data/Base_100milfilas.csv')
df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.2,0.773085,22,4,40,0.006919,-0.544676,AB,1998,...,0,200.0,0,INTERNET,28.199923,x11,1,1,0,0
2,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,AB,1548,...,1,200.0,0,INTERNET,11.234264,other,1,1,0,0
3,0,0.5,0.674870,-1,372,30,0.004406,30.300980,AA,2069,...,0,1500.0,0,INTERNET,6.336455,other,1,1,0,0
4,0,0.6,0.084088,-1,104,30,0.011475,-1.793004,AB,1972,...,1,200.0,0,INTERNET,4.502007,macintosh,1,1,0,0


In [4]:
print("-----------------(12). DIVIDO EL CONJUNTO DE DATOS EN ENTRENAMIENTO Y PRUEBA---------------\n")
# Divido el conjunto de datos en muestras de train y test
X = df.drop("fraud_bool", axis = 1)
y = df["fraud_bool"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

-----------------(12). DIVIDO EL CONJUNTO DE DATOS EN ENTRENAMIENTO Y PRUEBA---------------



,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
75220,0.4,0.583827,-1,68,20,0.006838,-0.623856,AB,1190,7284.168113,...,1,200.0,0,INTERNET,3.646187,other,0,1,0,5
48955,0.3,0.706788,-1,43,30,0.011727,-1.242702,AC,1648,1416.276017,...,0,1000.0,0,INTERNET,11.068744,other,1,1,0,3
44966,0.8,0.853240,-1,110,40,0.008390,-1.054796,AB,560,6943.630975,...,1,200.0,0,INTERNET,2.724917,windows,0,1,0,3
13568,0.2,0.477645,-1,84,20,0.001862,-0.966041,AB,5443,8737.892629,...,1,200.0,0,INTERNET,5.096423,linux,1,1,0,1
92727,0.4,0.403605,-1,107,40,0.004380,-1.324126,AC,769,2193.293927,...,0,200.0,0,INTERNET,4.399542,linux,0,1,0,7


In [5]:
print("-----------(13). IDENTIFICO Y ESCALO LAS VARIABLES NUMERICAS EXCLUYENDO LA VARIABLE OBJETIVO-----\n")
# Identifico las variables numéricas automáticamente, excluyendo 'fraud_bool' 
variables_numericas = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col]) and col != 'fraud_bool']

# Instancio el escalador
scaler = StandardScaler()

# Entreno el escalador con los datos de entrenamiento
scaler.fit(X_train[variables_numericas])

# Aplico el escalador en los datos de entrenamiento
X_train_final = scaler.transform(X_train[variables_numericas])
X_train_final = pd.DataFrame(X_train_final, index=X_train.index, columns=variables_numericas)

# Aplico el escalador en los datos de prueba
X_test_final = scaler.transform(X_test[variables_numericas])
X_test_final = pd.DataFrame(X_test_final, index=X_test.index, columns=variables_numericas)

# Muestro las primeras filas de los datos escalados de entrenamiento
print(X_train_final.head())

-----------(13). IDENTIFICO Y ESCALO LAS VARIABLES NUMERICAS EXCLUYENDO LA VARIABLE OBJETIVO-----

         income  name_email_similarity  prev_address_months_count  \
75220 -0.605160               0.347933                   -0.38469   
48955 -0.949709               0.769626                   -0.38469   
44966  0.773033               1.271878                   -0.38469   
13568 -1.294257              -0.016217                   -0.38469   
92727 -0.605160              -0.270136                   -0.38469   

       current_address_months_count  customer_age  days_since_request  \
75220                     -0.240803     -1.164783           -0.185708   
48955                     -0.523928     -0.354168           -0.184804   
44966                      0.234846      0.456447           -0.185421   
13568                     -0.059604     -1.164783           -0.186628   
92727                      0.200871      0.456447           -0.186163   

       intended_balcon_amount  zip_count_4w  ve

In [6]:
print("-----------(13). IDENTIFICO Y CODIFICO LAS VARIABLES CATEGORICAS------------------\n")
# Identificar las variables categóricas
columnas_categoricas = [col for col in X_train.columns if X_train[col].dtype == 'object']

# Instanciar el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # usar sparse_output en lugar de sparse

# Ajustar el OneHotEncoder a los datos de entrenamiento y transformar tanto train como test
X_train_categ = encoder.fit_transform(X_train[columnas_categoricas])
X_test_categ = encoder.transform(X_test[columnas_categoricas])

# Convertir a DataFrames y añadir al conjunto original
X_train_categ = pd.DataFrame(X_train_categ, columns=encoder.get_feature_names_out(columnas_categoricas), index=X_train.index)
X_test_categ = pd.DataFrame(X_test_categ, columns=encoder.get_feature_names_out(columnas_categoricas), index=X_test.index)

# Eliminar las variables categóricas originales del DataFrame
X_train = X_train.drop(columnas_categoricas, axis=1)
X_test = X_test.drop(columnas_categoricas, axis=1)

# Concatenar las nuevas variables dummy con el DataFrame original
X_train = pd.concat([X_train, X_train_categ], axis=1)
X_test = pd.concat([X_test, X_test_categ], axis=1)


-----------(13). IDENTIFICO Y CODIFICO LAS VARIABLES CATEGORICAS------------------



In [7]:
print("----(2). MODELO CON LOS DIFERENTES BOOSTING----\n")
model1_ada = AdaBoostClassifier(n_estimators=200, learning_rate=0.1, random_state=42)

----(2). MODELO CON LOS DIFERENTES BOOSTING----



In [8]:
print("----(3). ENTRENOS LOS DIFERENTES MODELOS BOOSTING----\n")
model1_ada.fit(X_train, y_train)

----(3). ENTRENOS LOS DIFERENTES MODELOS BOOSTING----



AdaBoostClassifier(learning_rate=0.1, n_estimators=200, random_state=42)

In [9]:
print("----(4). REALIZO PREDICCION EN EL TEST----\n")
ada_y_pred_test = model1_ada.predict(X_test)
ada_y_pred_test

----(4). REALIZO PREDICCION EN EL TEST----



array([0, 0, 0, ..., 0, 0, 0], shape=(20000,))

In [10]:
print("----(5). REALIZO PREDICCION EN EL TRAIN----\n")
ada_y_pred_train = model1_ada.predict(X_train)
ada_y_pred_train

----(5). REALIZO PREDICCION EN EL TRAIN----



array([0, 0, 0, ..., 0, 0, 0], shape=(80000,))

In [11]:
print("----(6). SACO LAS METRICAS TANTO EN TRAIN Y EL TEST PARA LOS MODELOS BOOSTING----\n")

ada_accuracy_test = accuracy_score(y_test, ada_y_pred_test)
ada_accuracy_train = accuracy_score(y_train, ada_y_pred_train)

----(6). SACO LAS METRICAS TANTO EN TRAIN Y EL TEST PARA LOS MODELOS BOOSTING----



In [12]:
print('Mètricas del AdaBoost')
print("Accuracy Test AdaBoost Inicial: ", ada_accuracy_test)
print("Accuracy Train AdaBoost Inicial: ", ada_accuracy_train)

Mètricas del AdaBoost
Accuracy Test AdaBoost Inicial:  0.8957
Accuracy Train AdaBoost Inicial:  0.8948


In [13]:
# Definir la ruta donde se guardará el modelo
ruta_modelo = '/workspaces/fraud/models/modelo_AdaBoost_inicial.pkl'

# Verificar si el directorio existe, si no, crearlo
directorio = os.path.dirname(ruta_modelo)
if not os.path.exists(directorio):
    os.makedirs(directorio)

# Guardar el modelo AdaBoost
joblib.dump(model1_ada, ruta_modelo)

print(f"Modelo guardado en: {ruta_modelo}")

Modelo guardado en: /workspaces/fraud/models/modelo_AdaBoost_inicial.pkl
